# Sigmoid
# TruncatedNormal0.1x_4

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import numpy as np
import os, shutil
from scipy.stats import truncnorm
from scipy.stats import norm

def truncn(shape, fan_in, fan_out=0, k=2):
    N = (fan_in + fan_out)/2
    
    z = norm.cdf(k) - norm.cdf(-k)
    
    constant = (1 + (-k*norm.pdf(-k) -k*norm.pdf(k))/z - ((norm.pdf(-k)-norm.pdf(k))/z)**2)
    
    sd = np.sqrt(1/N/constant)
    
    return tf.random.truncated_normal(
        shape = shape,
        mean = 0,
        stddev = sd,
        dtype=tf.float64,
        seed=None,
        name=None)

os.chdir('C:/experiment/relu') # 현재 디렉토리를 설정
base_dir = 'C:/experiment/relu/truncatednormal0.5' # 베이스 디렉토리
if os.path.exists(base_dir):  
    shutil.rmtree(base_dir) # 베이스 디렉토리가 존재하면 베이스디렉토리를 삭제
os.mkdir(base_dir)

(train_x, train_y), (test_x, test_y) = mnist.load_data()

train_x = train_x.reshape([-1,28*28])
train_x = train_x / 255
mean = np.mean(train_x); var = np.var(train_x);
train_x = (train_x - mean) / np.sqrt(var)

test_x  =  test_x.reshape([-1,28*28])
test_x = test_x/ 255
test_x = (test_x - mean) / np.sqrt(var)

train_y = tf.keras.utils.to_categorical(train_y)
test_y  = tf.keras.utils.to_categorical(test_y)

batch_size = 100.
total_batch = int(len(train_x) / batch_size)

X = tf.placeholder(tf.float64, [None, 784], name = 'inputs' )
Y = tf.placeholder(tf.float64, [None, 10] , name = 'labels')

costs = []

with tf.name_scope('Input_Layer') as scope:
    with tf.name_scope("Layer_1"):
        W1 = tf.Variable(truncn([784,1000], 784, 1000, k=0.5))
        L1 = tf.nn.relu(tf.matmul(X, W1))
        tf.summary.histogram("L1", L1)
        
    with tf.name_scope("Layer_2"):
        W2 = tf.Variable(truncn([1000,1000], 1000, 1000, k=0.5))
        L2 = tf.nn.relu(tf.matmul(L1, W2))
        tf.summary.histogram("L2", L2)
        
    with tf.name_scope("Layer_3"):
        W3 = tf.Variable(truncn([1000,1000], 1000, 1000, k=0.5))
        L3 = tf.nn.relu(tf.matmul(L2, W3))
        tf.summary.histogram("L3", L3)

    with tf.name_scope("Layer_4"):
        W4 = tf.Variable(truncn([1000,1000], 1000, 1000, k=0.5))
        L4 = tf.nn.relu(tf.matmul(L3, W4))
        tf.summary.histogram("L4", L4)

    with tf.name_scope("Output"):
        W_F = tf.Variable(truncn([1000,10], 1000, 10, k=0.5))
        score = tf.matmul(L4, W_F)

with tf.name_scope('Cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=score, labels=Y))
    tf.summary.scalar('Cost', cost)

with tf.name_scope('Training'):
    optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

with tf.name_scope('Accuracy'):
    is_correct = tf.equal(tf.argmax(score, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))*100
    tf.summary.scalar('Accuracy', accuracy)
    
merged = tf.summary.merge_all()

with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(base_dir + '/train', graph = tf.get_default_graph())
    test_writer = tf.summary.FileWriter(base_dir + '/test', graph = tf.get_default_graph())
    sess.run(tf.global_variables_initializer())

    for epoch in range(30):
        total_cost = 0
        
        for i in range(total_batch):
            batch_x = train_x[(100*i-100):(100*i-1)]
            batch_y = train_y[(100*i-100):(100*i-1)]
            
            cost_val, _ = sess.run([cost, optimizer], feed_dict={X: batch_x, Y: batch_y})
            summary  = sess.run(merged, feed_dict={X: batch_x, Y: batch_y})
            
            train_writer.add_summary(summary, i)
            total_cost += cost_val
            
        costs.append(total_cost / total_batch)
            
        print('Epoch:', '%04d' % (epoch + 1),
              'Avg. cost =', '{:.6f}'.format(total_cost / total_batch))
        
    summary2 = sess.run(merged, feed_dict={X: test_x, Y: test_y})
    test_writer.add_summary(summary2)
    
    print('Opitimization Complete!')
    
    print('Accuracy:', sess.run(accuracy,
                        feed_dict={X: test_x,
                                   Y: test_y}))
    
    print(costs)

C:\Users\woohy\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Epoch: 0001 Avg. cost = 5.051903
Epoch: 0002 Avg. cost = 0.155952
Epoch: 0003 Avg. cost = 0.045267
Epoch: 0004 Avg. cost = 0.015142
Epoch: 0005 Avg. cost = 0.006209
Epoch: 0006 Avg. cost = 0.003671
Epoch: 0007 Avg. cost = 0.002626
Epoch: 0008 Avg. cost = 0.002089
Epoch: 0009 Avg. cost = 0.001767
Epoch: 0010 Avg. cost = 0.001539
Epoch: 0011 Avg. cost = 0.001366
Epoch: 0012 Avg. cost = 0.001230
Epoch: 0013 Avg. cost = 0.001120
Epoch: 0014 Avg. cost = 0.001028
Epoch: 0015 Avg. cost = 0.000950
Epoch: 0016 Avg. cost = 0.000884
Epoch: 0017 Avg. cost = 0.000826


Epoch: 0001 Avg. cost = 2.742168
Epoch: 0002 Avg. cost = 1.123606
Epoch: 0003 Avg. cost = 0.802217
Epoch: 0004 Avg. cost = 0.632079
Epoch: 0005 Avg. cost = 0.520855
Epoch: 0006 Avg. cost = 0.440322
Epoch: 0007 Avg. cost = 0.378483
Epoch: 0008 Avg. cost = 0.329225
Epoch: 0009 Avg. cost = 0.289017
Epoch: 0010 Avg. cost = 0.255586
Epoch: 0011 Avg. cost = 0.227391
Epoch: 0012 Avg. cost = 0.203360
Epoch: 0013 Avg. cost = 0.182705
Epoch: 0014 Avg. cost = 0.164826
Epoch: 0015 Avg. cost = 0.149259
Epoch: 0016 Avg. cost = 0.135636
Epoch: 0017 Avg. cost = 0.123658
Epoch: 0018 Avg. cost = 0.113084
Epoch: 0019 Avg. cost = 0.103710
Epoch: 0020 Avg. cost = 0.095373
Epoch: 0021 Avg. cost = 0.087934
Epoch: 0022 Avg. cost = 0.081280
Epoch: 0023 Avg. cost = 0.075313
Epoch: 0024 Avg. cost = 0.069948
Epoch: 0025 Avg. cost = 0.065111
Epoch: 0026 Avg. cost = 0.060741
Epoch: 0027 Avg. cost = 0.056782
Epoch: 0028 Avg. cost = 0.053190
Epoch: 0029 Avg. cost = 0.049923
Epoch: 0030 Avg. cost = 0.046948
Epoch: 0031 Avg. cost = 0.044233
Epoch: 0032 Avg. cost = 0.041754
Epoch: 0033 Avg. cost = 0.039487
Epoch: 0034 Avg. cost = 0.037411
Epoch: 0035 Avg. cost = 0.035508
Epoch: 0036 Avg. cost = 0.033760
Epoch: 0037 Avg. cost = 0.032152
Epoch: 0038 Avg. cost = 0.030670
Epoch: 0039 Avg. cost = 0.029302
Epoch: 0040 Avg. cost = 0.028036
Epoch: 0041 Avg. cost = 0.026864
Epoch: 0042 Avg. cost = 0.025777
Epoch: 0043 Avg. cost = 0.024767
Epoch: 0044 Avg. cost = 0.023827
Epoch: 0045 Avg. cost = 0.022950
Epoch: 0046 Avg. cost = 0.022131
Epoch: 0047 Avg. cost = 0.021365
Epoch: 0048 Avg. cost = 0.020648
Epoch: 0049 Avg. cost = 0.019973
Epoch: 0050 Avg. cost = 0.019339
Opitimization Complete!
Accuracy: 86.77
[2.742168488591967, 1.1236061095835477, 0.802217125209002, 0.6320792095616717, 0.5208549105043742, 0.4403220865792594, 0.3784829352308924, 0.3292248116443922, 0.2890172234175097, 0.25558621086065264, 0.22739144556852645, 0.20335969878479618, 0.18270474266467604, 0.1648263625299688, 0.1492591568276661, 0.13563556257783158, 0.12365842584995623, 0.11308376849456771, 0.10371035250606543, 0.09537256061103862, 0.08793388681255, 0.08127999217246946, 0.07531285387388786, 0.06994763530923705, 0.06511120626555457, 0.060740778331246165, 0.05678247410374234, 0.05318994328252955, 0.049923163227481, 0.04694755607561104, 0.044233214465350376, 0.04175404826806922, 0.03948694754473134, 0.037411134299520615, 0.03550777236836479, 0.03375977680064616, 0.0321517246322341, 0.03066979143825165, 0.029301669996058415, 0.0280364537490891, 0.026864484754333742, 0.02577717829413246, 0.024766846169866904, 0.023826543382104805, 0.02294995513236337, 0.022131326204409243, 0.021365421662642055, 0.020647502748729084, 0.019973304537292996, 0.019339008300758256]

Epoch: 0001 Avg. cost = 2.799177
Epoch: 0002 Avg. cost = 1.134641
Epoch: 0003 Avg. cost = 0.810335
Epoch: 0004 Avg. cost = 0.643111
Epoch: 0005 Avg. cost = 0.534156
Epoch: 0006 Avg. cost = 0.455195
Epoch: 0007 Avg. cost = 0.394378
Epoch: 0008 Avg. cost = 0.345674
Epoch: 0009 Avg. cost = 0.305635
Epoch: 0010 Avg. cost = 0.272120
Epoch: 0011 Avg. cost = 0.243677
Epoch: 0012 Avg. cost = 0.219266
Epoch: 0013 Avg. cost = 0.198126
Epoch: 0014 Avg. cost = 0.179686
Epoch: 0015 Avg. cost = 0.163511
Epoch: 0016 Avg. cost = 0.149258
Epoch: 0017 Avg. cost = 0.136650
Epoch: 0018 Avg. cost = 0.125457
Epoch: 0019 Avg. cost = 0.115484
Epoch: 0020 Avg. cost = 0.106567
Opitimization Complete!
Accuracy: 84.47
[2.7991766759782326, 1.134641017101305, 0.8103350093149833, 0.6431112028173173, 0.5341555681844551, 0.4551953393829066, 0.3943784540003952, 0.34567369917406554, 0.30563505654667006, 0.2721197594720991, 0.24367653391569685, 0.2192659779489422, 0.19812607198811621, 0.17968634830149138, 0.16351074709353455, 0.14925790706063258, 0.13665048459768123, 0.1254571344420934, 0.11548382527390152, 0.10656727839690366]